In [4]:
import requests
import pandas as pd
import time
from tqdm import tqdm
import plotly.express as px
import json

In [5]:
tier_list = ['challenger', 'grandmaster', 'master',
             'diamond',  'platinum', 'gold', 'silver', 'bronze', 'iron']

stats_name = ['win_rate', 'pick_rate', 'ban_rate']

In [6]:
# 데이터 수집
def get_data(tier):
    try:
        time.sleep(1)
        url = 'https://www.op.gg/_next/data/jJBmp4Iu3tyDIZFxaXguO/champions.json?region=kr'
        response = requests.get(url, headers={'User-agent': 'Mozilla/5,0'},
                                params={'tier': tier, 'patch': '12.10'})
        data = response.json()['pageProps']['championMetaList']
        return data
    except:
        raise Exception(response.status_code)


# 데이터 저장하기
def save_json():
    for tier in tqdm(tier_list):
        data = get_data(tier)

        data_path = f'data/12.10_{tier}_data.json'
        with open(data_path, 'w') as f:
            json.dump(data, f)


save_json()

100%|██████████| 9/9 [00:11<00:00,  1.23s/it]


In [1]:
# 데이터프레임 만들기
def make_dataframe():
    df_list = []
    for tier in tqdm(tier_list):
        file_name = f'12.10_{tier}_'

        data_path = f'data/{file_name}data.json'
        with open(data_path) as f:
            data = json.load(f)

        # make dict
        data_dict = {}
        for d in tqdm(data):
            champion = d['name']
            for p in d['positions']:
                stats_list = []
                position = p['name']
                for n in stats_name:
                    stats_list.append(p['stats'][n] * 100)
                data_dict[(champion, position)] = stats_list

        # make df
        df = pd.DataFrame(data_dict).T.reset_index()
        df.columns = ['champion', 'position',
                      'win_rate', 'pick_rate', 'ban_rate']
        df.insert(loc=0, column='tier', value=tier)
        df_list.append(df)
    df = pd.concat(df_list)
    # 라이엇의 오타 해결
    df.loc[df['position'] == 'ALL', ['position']] = 'ADC'
    # save df
    df.to_csv('dataframe.csv', index=False)

In [7]:
make_dataframe()

100%|██████████| 9/9 [00:00<00:00, 37.28it/s]


In [8]:
df = pd.read_csv('dataframe.csv')

fig = px.scatter(df, x='pick_rate',
                 y='ban_rate', color='position', animation_frame='tier', hover_data=['champion'], range_x=[-10, 60], range_y=[-10, 60])
fig.show()